# 필요한 페키지 임포트, 함수 정의 등

In [ ]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 함수 정의

# folder_path에 url에서 받아온 pdf를 filename으로 저장하는 함수
def download_pdf(url, folder_path, filename, party):
    response = requests.get(url)
    if response.status_code == 200:
        #filename = url.split("/")[-1]
        file_path = os.path.join(folder_path+party, filename)
        with open(file_path, 'wb') as f:
            f.write(response.content)
        print(f"Downloaded: {filename}")
    else:
        print(f"Failed to download: {url}")


# 실제 크롤링을 수행하는 함수
def scrape_candidates(url, folder_path, parties, order):

    order = order
    page = 1

    while (True) :
        if (page%10 == 0) :
            print(order, page)

        response = requests.get(url.format(page, order))
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser').select_one('#content > div.searchList > ul')

            if soup == None :
                return

            for li in soup.find_all('li', class_='clear'):
                name = li.find('strong').text.strip()
                info = li.find('p', class_='l10 t10').text.strip()
                party = info.split("|")[-2].strip()

                if party in parties:
                    pdf_links = li.find_all('a', href=lambda href: href)
                    for link in pdf_links:
                        pdf_url = urljoin(url, link['href'])
                        # 모든 다운로드 링크 확인
                        if "벽보" in link.img['alt'] and "viewPdf" not in pdf_url:
                            filename = info + " | " + name
                            download_pdf(pdf_url, folder_path, filename, party)
                            break

        page += 1

In [ ]:
# 스크래핑할 선관위 도서관 링크

총선 = "https://library.nec.go.kr/neweps/3/1/paperSearch.do?chs=&page={0}&epType=&start_file_sn=&end_file_sn=&svc_path_nm=&epid=&candidate_nm=&elect_ymd=&epdata_id=&ctl_no_id=&elect_sn=&party_nm=&win_yn=&elecDe=&turnDe=%B1%B9%C8%B8%C0%C7%BF%F8%BC%B1%B0%C5&elecType=20&code_id=20++&elecTypeMax=30&turnType=EPS0420++&cityType=EPS0420{1}&guType=&sign_id={1}&sido_nm=&elect_region_nm=&fieldName=candidate_nm&category=tms120tbl&myScraps=&kwd=&elecTypes=20&electionType=on&electionTurn=on&pageSize=10&order=elect_ymd&order_sort=desc&m_score=%B8%C5%BF%EC%B8%B8%C1%B7"
광역 = "https://library.nec.go.kr/neweps/3/1/paperSearch.do?chs=&page={0}&epType=&start_file_sn=&end_file_sn=&svc_path_nm=&epid=&candidate_nm=&elect_ymd=&epdata_id=&ctl_no_id=&elect_sn=&party_nm=&win_yn=&elecDe=&turnDe=%B1%A4%BF%AA%B4%DC%C3%BC%C0%E5%BC%B1%B0%C5&elecType=30&code_id=30++&elecTypeMax=50&turnType=EPS0430++&cityType=EPS04300{1}&guType=&sign_id={1}&sido_nm=&elect_region_nm=&fieldName=candidate_nm&category=tms120tbl&myScraps=&kwd=&elecTypes=30&electionType=on&electionTurn=on&pageSize=10&order=elect_ymd&order_sort=desc&m_score=%B8%C5%BF%EC%B8%B8%C1%B7"
기초 = "https://library.nec.go.kr/neweps/3/1/paperSearch.do?chs=&page={0}&epType=&start_file_sn=&end_file_sn=&svc_path_nm=&epid=&candidate_nm=&elect_ymd=&epdata_id=&ctl_no_id=&elect_sn=&party_nm=&win_yn=&elecDe=&turnDe=%B1%E2%C3%CA%B4%DC%C3%BC%C0%E5%BC%B1%B0%C5&elecType=30&code_id=40++&elecTypeMax=50&turnType=EPS0440++&cityType=EPS04400{1}&guType=&sign_id={1}&sido_nm=&elect_region_nm=&fieldName=candidate_nm&category=tms120tbl&myScraps=&kwd=&elecTypes=30&electionType=on&electionTurn=on&pageSize=10&order=elect_ymd&order_sort=desc&m_score=%B8%C5%BF%EC%B8%B8%C1%B7"

# 스크래핑 실행 코드

In [ ]:
# 총선 벽보 pdf 파일 스크래핑

url = 총선
orders = [21, 20, 19, 18, 17, 16]
party_list = [["더불어민주당", "국민의힘"],
              ["더불어민주당", "새누리당", "국민의당"],
              ["새누리당", "민주통합당"],
              ["한나라당", "통합민주당"],
              ["열린우리당", "한나라당"],
              ["한나라당", "새천년민주당"]] # 교섭단체(20석 이상) 정당만

folder_path = "" # pdf파일을 저장할 폴더 경로

for i in range(len(orders)) :
    new_path = folder_path + str(orders[i]) + "대 총선 "
    scrape_candidates(url, new_path, party_list[i], orders[i])

In [ ]:
# 지선 광역 벽보 pdf 파일 스크래핑

url = 광역
orders = [8, 7, 6, 5, 4, 3]
party_list = [["더불어민주당", "국민의힘"],
              ["더불어민주당", "자유한국당"],
              ["새정치민주연합", "새누리당"],
              ["민주당", "한나라당"],
              ["민주당", "열린우리당", "한나라당"],
              ["새천년민주당", "한나라당"]]

folder_path = ""

for i in range(len(orders)) :
    new_path = folder_path + str(orders[i]) + "대 지선 광역 "
    scrape_candidates(url, new_path, party_list[i], orders[i])

In [ ]:
# 지선 기초 벽보 pdf 파일 스크래핑

url = 기초
orders = [8, 7, 6, 5, 4, 3]
party_list = [["더불어민주당", "국민의힘"],
              ["더불어민주당", "자유한국당"],
              ["새정치민주연합", "새누리당"],
              ["민주당", "한나라당"],
              ["민주당", "열린우리당", "한나라당"],
              ["새천년민주당", "한나라당"]]

folder_path = ""

for i in range(len(orders)) :
    new_path = folder_path + str(orders[i]) + "대 지선 기초 "
    scrape_candidates(url, new_path, party_list[i], orders[i])